# ✈️ FlightOnTime — Predição de Atrasos em Voos

O **FlightOnTime** é um projeto de *Data Science* e *Back-End* voltado para prever a probabilidade de um voo **decolar no horário ou com atraso**, utilizando dados históricos de aviação civil.  
Ele faz parte de um desafio educacional cujo objetivo é construir um **MVP preditivo** capaz de apoiar passageiros, companhias aéreas e aeroportos na tomada de decisão.

## Descrição do projeto

### 🎯 Objetivo do Projeto

Criar um modelo de classificação binária que, a partir de informações de um voo — como companhia aérea, origem, destino, horário e distância — estima se ele será **Pontual (0)** ou **Atrasado (1)**, retornando também a probabilidade associada.

---

### 🧠 Pipeline de Data Science

O notebook conduz as principais etapas do fluxo de ciência de dados:

1. **ETL e limpeza dos dados**  
   - Carregamento, padronização e enriquecimento das informações de voos.

2. **Engenharia de variáveis (Feature Engineering)**  
   - Extração de atributos relevantes, como:  
     hora do voo, dia da semana, distância, aeroportos, companhia aérea.

3. **Modelagem preditiva**  
   - Testes com algoritmos supervisionados (ex.: Logistic Regression, Random Forest).

4. **Avaliação do modelo**  
   - Métricas como Acurácia, Precisão, Recall e F1-score.

5. **Exportação do modelo treinado**  
   - Serialização via `joblib` para uso pela API de Back-End.

---

### 🛠️ Integração com a API

O modelo gerado neste notebook será consumido por uma **API REST em Java (Spring Boot)**, que disponibilizará o endpoint:

```
POST /predict
```

Retornando:

```json
{
  "previsao": "Atrasado",
  "probabilidade": 0.78
}
```

---

### 📌 Sobre o Dataset

O projeto utiliza um conjunto de dados contendo informações reais de voos, incluindo:

- companhia aérea  
- aeroporto de origem e destino  
- data e horário de partida  
- distância do trecho  
- status de atraso  

As funções de ETL foram previamente desenvolvidas para garantir limpeza, consistência e rastreabilidade.

## 🚀 Início do Notebook

A partir daqui, o notebook executa o fluxo completo de preparação, análise e modelagem, resultando em um modelo leve, funcional e pronto para produção no MVP do FlightOnTime.

### Imports Globais

In [ ]:
import pandas as pd

from etl.etl import processar_dados, carregar_dados
from sklearn.model_selection import train_test_split

from utils.visualize_y import visualize_y
from utils.export_model import export_model
from utils.document_model import document_model
from utils.evaluate_model import evaluate_model
from utils.plot_automl_results import plot_algorithm_selection, plot_adaptive_sampling, plot_feature_selection, plot_model_tuning
from utils.experiment_automl_pipelines import run_experiments, evaluate_experimental_models, load_experimental_models

import automlx
from automlx import init # type: ignore

### Preprocessamento e Carga

Execute as funções abaixo para carregar os dados de fonte local ou para baixar, preprocessar e salvar os dados a partir da base de dados de vôos da ANAC (Agência Nacional de Aviação Civil).  

- Os dados são salvos em ./dados/
- A função `processar_dados` acessa toda a base de dados da ANAC a partir de Janeiro/2018 até Outubro/2025.
- O processo de ETL completo é executado durante o processo.
- A engenharia de features já foi executada durante o processo, gerando a coluna "Atrasado" (Y) 

Para mais informações sobre o processo, consulte a [Documentação](./etl/README.md) do módulo etl.

#### Preprocessar com origem externa

Baixa e executa o preprocessamento em tempo de execução através da base de dados abertos da ANAC.  
Link: https://sistemas.anac.gov.br/dadosabertos/Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/Voo%20Regular%20Ativo%20%28VRA%29/

In [ ]:
# Código comentado por segurança
# Descomente a linha abaixo para executar o pipeline ETL completo

# df = processar_dados()

#### Preprocessar com origem local

Exemplo de nome de arquivo local:  
```python
filename = "dados_voos_20250101_153000"
```

- Não necessita informar o caminho do arquivo e extensão.  
- Caminho default: ./data
- Extensão default: .parquet

In [ ]:
filename = "dados_voos_20260108_193425"
df = carregar_dados(filename=filename)

#### Visualizações

In [ ]:
df.head()

##### Verificando o tamanho do dataframe

In [ ]:
df.shape

##### Verificando os tipos das colunas

In [ ]:
df.info()

#### Verificando ETL

In [ ]:
def check_nulls(df: pd.DataFrame, col: str) -> None:
    nulls_notice = f'Coluna "{col}" possui {df[col].isna().sum()} valores nulos.'
    print(nulls_notice)
    return

def check_uniques(df: pd.DataFrame, col: str) -> None:
    print(f'Coluna "{col}":')
    print(f'{df[col].unique()}\n')
    return

##### Verificando Nulos

In [ ]:
for col in df.columns:
    check_nulls(df, col)

##### Verificando Uniques

In [ ]:
for col in df.columns:
    check_uniques(df, col)

### Visualizando (Y)

Aqui analisamos a distribuição dos vôos atrasados no histórico de vôos observados.  

- Aproximadamente 2.5 milhões de vôos foram pontuais.  
- Aproximadamente 1.4 milhões de vôos atrasaram, totalizando aproximadamente 35.45% dos vôos observados.  

In [ ]:
visualize_y(df, y_col_name='Atrasado')

### Separando X e Y

Aqui separaremos as features (X) da variável alvo (Y).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['Atrasado']),
    df['Atrasado'],
    test_size=0.3,
    random_state=42,
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
X_train.head()

In [ ]:
y_train.head()

### Modelagem

Para o processo de modelagem, decidimos usar a moderna biblioteca [Oracle AutoMLx](https://docs.oracle.com/en-us/iaas/tools/automlx/latest/latest/index.html), que automatiza os processos de `Seleção de Algoritmo`, `Sampling Adaptativo`, `Seleção de Features` e `Ajuste de Hiperparâmetros`, resultando num processo facilitado e com melhores métricas no modelo final.  

In [ ]:
init(engine='local')

#### Referência de Pipeline:

In [ ]:
# Referência de Pipeline:
# Deletar depois **

# custom_pipeline = automlx.Pipeline(
#     task='classification',
#     model_list=[                 # Specify the models you want the AutoMLx to consider
#         'LogisticRegression',
#         'LGBMClassifier',
#         'GaussianNB'
#     ],
#     n_algos_tuned=2,             # Choose how many models to tune
#     min_features=[               # Specify minimum features to force the model to use. It can take 3 possible types of values:
#         'native-country',        # If int, 0 < min_features <= n_features,
#         'marital-status',              # If float, 0 < min_features <= 1.0, 1.0 means disabling feature selection
#         'education-num'          # If list, names of features to keep, for example ['a', 'b'] means keep features 'a' and 'b'
#     ],
#     adaptive_sampling=False,     # Disable or enable Adaptive Sampling step. Default to `True`
#     preprocessing=True,          # Disable or enable Preprocessing step. Default to `True`
#     search_space={               # You can specify the hyper-parameters and ranges we search
#         'LGBMClassifier': {
#             'learning_rate': {'range': [0.01, 10], 'type': 'continuous'},
#             'boosting_type': {'range': ['gbdt', 'dart'], 'type': 'categorical'},
#         },
#     },
#     max_tuning_trials=2,         # The maximum number of tuning trials. Can be integer or Dict (max number for each model)
#     score_metric='f1_macro',     # Any scikit-learn metric or a custom function
# )

# est1 = custom_pipeline.fit(
#     X_train,
#     y_train,
#     # X_val,            # Optionally, provide validation data to skip internal CV
#     # y_val,
#     time_budget= 20,    # Specify time budget in seconds
#     cv='auto'           # Automatically pick a good cross-validation (cv) strategy for the user's dataset.
#                         # Ignored if X_valid and y_valid are provided.
#                         # Can also be:
#                         #   - An integer (For example, to use 5-fold cross validation)
#                         #   - A list of data indices to use as splits (for advanced, such as time-based splitting)
# )

#### Experimentação - Avaliação de Pipelines

Nesta etapa, experimentamos diferentes configurações de `Pipeline` do Oracle AutoMLx com o objetivo de **compreender o comportamento do nosso dataset em tarefas de classificação**, bem como **avaliar o impacto de diferentes escolhas de modelagem** (tipo de modelo, amostragem, seleção de features e tuning).

Cada experimento foi desenhado para responder a uma **pergunta específica**, evitando combinações arbitrárias e permitindo interpretações claras dos resultados.

---

##### 1. Baseline Linear — Regressão Logística

Geramos inicialmente um modelo baseline de **Regressão Logística**, com pré-processamento automático habilitado e sem técnicas de reamostragem.

Este experimento tem como objetivos:
- Avaliar o quão bem um modelo **linear** consegue capturar o padrão de atrasos nos voos;
- Servir como **âncora de interpretabilidade**, uma vez que modelos lineares facilitam a análise do efeito marginal das variáveis;
- Estabelecer um **limite inferior de performance** para comparação com modelos mais complexos.

---

##### 2. Baseline Não Linear — LightGBM sem Tuning

Em seguida, treinamos um modelo **LightGBM** utilizando as configurações padrão, sem ajuste de hiperparâmetros e sem amostragem adaptativa.

Este experimento busca:
- Avaliar o ganho de performance ao introduzir **não linearidades e interações entre variáveis**;
- Estabelecer um **baseline robusto de modelos baseados em árvores**;
- Comparar diretamente com a Regressão Logística para verificar se o problema exige maior capacidade de modelagem.

---

##### 3. Impacto da Amostragem Adaptativa

Mantendo o LightGBM sem tuning, ativamos a funcionalidade de **adaptive sampling** do AutoMLx.

O objetivo deste experimento é:
- Verificar se a leve desproporção entre as classes (≈65% / 35%) se beneficia de técnicas de reamostragem;
- Avaliar o impacto da amostragem adaptativa na **capacidade discriminativa global** do modelo (ROC AUC);
- Identificar possíveis trade-offs entre recall da classe minoritária e calibração de probabilidades.

---

##### 4. Forçando Features Relevantes ao Domínio

Neste experimento, utilizamos novamente o LightGBM, mas **forçando a inclusão de variáveis consideradas críticas pelo conhecimento de domínio**, como:
- Empresa Aérea  
- Aeródromo de Origem  
- Aeródromo de Destino  

A motivação é:
- Avaliar se o mecanismo automático de seleção de features do AutoMLx está descartando informações relevantes;
- Testar se a imposição dessas variáveis melhora a capacidade de generalização do modelo;
- Validar a confiança no processo automatizado de feature selection.

---

##### 5. Tuning Leve de Hiperparâmetros do LightGBM

Aqui realizamos um **ajuste leve de hiperparâmetros** do LightGBM, visando comparar o modelo ajustado com o modelo baseline `2`.  

Este experimento busca responder:
- Se pequenos ajustes em hiperparâmetros estruturais (como profundidade, número de folhas e taxa de aprendizado) trazem ganhos mensuráveis;
- Se o custo computacional do tuning é justificável dado o grande volume de dados;
- Se as configurações padrão já se encontram próximas do ótimo para este problema.

---

##### 6. Modelo de Sanidade — Naive Bayes Gaussiano

Por fim, treinamos um modelo **Gaussian Naive Bayes**, que assume independência condicional entre as variáveis.

Este experimento não visa competir em performance, mas sim:
- Atuar como um **teste de sanidade**;
- Avaliar se o problema pode ser explicado por relações estatísticas simples;
- Confirmar a necessidade de modelos capazes de capturar interações complexas.

---

Ao final desses experimentos, temos uma visão clara sobre:
- A complexidade intrínseca do problema;
- O impacto real de técnicas de automação (sampling, feature selection, tuning);
- O melhor equilíbrio entre performance, custo computacional e interpretabilidade.

In [ ]:
PIPELINE_CONFIGS = {
    # --------------------------------------------------
    # 1) Linear baseline (interpretability anchor)
    # --------------------------------------------------
    "01_baseline_logreg": {
        "task": "classification",
        "model_list": ["LogisticRegression"],
        "preprocessing": True,
        "adaptive_sampling": False,
        "score_metric": "roc_auc",
    },

    # --------------------------------------------------
    # 2) Tree baseline (no tuning, full data)
    # --------------------------------------------------
    "02_baseline_lgbm": {
        "task": "classification",
        "model_list": ["LGBMClassifier"],
        "n_algos_tuned": 1,
        "max_tuning_trials": 1,
        "preprocessing": True,
        "adaptive_sampling": False,
        "score_metric": "roc_auc",
    },

    # --------------------------------------------------
    # 3) Effect of adaptive sampling
    # --------------------------------------------------
    "03_lgbm_with_sampling": {
        "task": "classification",
        "model_list": ["LGBMClassifier"],
        "n_algos_tuned": 1,
        "max_tuning_trials": 1,
        "adaptive_sampling": True,
        "score_metric": "roc_auc",
    },

    # --------------------------------------------------
    # 4) Force domain-important features
    # (tests AutoMLx feature selection behavior)
    # --------------------------------------------------
    "04_lgbm_forced_features": {
        "task": "classification",
        "model_list": ["LGBMClassifier"],
        "n_algos_tuned": 1,
        "max_tuning_trials": 1,
        "min_features": [
            "Empresa Aérea",
            "Aeródromo Origem",
            "Aeródromo Destino",
        ],
        "adaptive_sampling": False,
        "score_metric": "roc_auc",
    },

    # --------------------------------------------------
    # 5) Light Hyperparameter tuning
    # --------------------------------------------------
    "05_lgbm_light_tuning": {
        "task": "classification",
        "model_list": ["LGBMClassifier"],
        "adaptive_sampling": False,
        "max_tuning_trials": 5,
        "score_metric": "roc_auc",
    },

    # --------------------------------------------------
    # 6) Naive Bayes sanity check
    # --------------------------------------------------
    "06_nb_sanity": {
        "task": "classification",
        "model_list": ["GaussianNB"],
        "preprocessing": True,
        "adaptive_sampling": False,
        "score_metric": "roc_auc",
    },
}


##### Realizando experimentação

Esta funcionalidade permite realizar a experimentação de pipelines conforme as configurações contidas no dicionário `PIPELINE_CONFIGS`. Após a experimentação, os modelos são salvos em ./models/experimental_models.  

In [ ]:
# Código comentado por segurança
# Descomente a linha abaixo para executar a experimentação dos pipelines AutoMLx

experimental_models = run_experiments(pipeline_configs=PIPELINE_CONFIGS, X_train=X_train, y_train=y_train)

##### Carregar Modelos Experimentais

Esta funcionalidade permite carregar modelos experimentais previamente treinados e salvos a partir de um dicionário `PIPELINE_CONFIGS`.

In [ ]:
experimental_models = load_experimental_models(PIPELINE_CONFIGS)

##### Avaliando Modelos Experimentais

In [ ]:
evaluate_experimental_models(
    experimental_models=experimental_models,
    X_test=X_test,
    y_test=y_test,
)

##### Insights

| Experimento           | ROC AUC    | Principal insight                                   |
| --------------------- | ---------- | --------------------------------------------------- |
| baseline_logreg       | ~0.600     | Modelos lineares são insuficientes                  |
| baseline_lgbm         | ~0.686     | Estrutura não linear é essencial                   |
| lgbm_with_sampling    | ~0.687     | A amostragem adaptativa tem efeito desprezível     |
| lgbm_forced_features  | ~0.686     | A seleção automática de features do AutoMLx é confiável |
| lgbm_light_tuning     | **~0.699** | Tuning leve gera ganhos reais de performance        |
| nb_sanity             | ~0.601     | O sinal não é trivial                               |

Os experimentos realizados fornecem uma visão clara sobre a natureza do problema e sobre as escolhas mais adequadas de modelagem.

- Em primeiro lugar, a comparação entre a Regressão Logística e os modelos baseados em árvores evidencia que **modelos lineares não são suficientes** para capturar a complexidade do fenômeno de atraso de voos. O ganho expressivo de ROC AUC ao utilizar o LightGBM indica a presença de **relações não lineares e interações relevantes** entre variáveis categóricas, numéricas e temporais.

- A avaliação do uso de **adaptive sampling** mostrou impacto praticamente nulo na performance global do modelo. Isso sugere que a leve desproporção entre as classes (aproximadamente 65% / 35%) não compromete o aprendizado, e que o LightGBM consegue lidar adequadamente com a distribuição original dos dados, tornando desnecessária a introdução de técnicas adicionais de reamostragem.

- Ao forçar a inclusão de variáveis consideradas críticas pelo conhecimento de domínio, não observamos melhorias de desempenho. Esse resultado indica que o mecanismo automático de seleção de features do Oracle AutoMLx é confiável e não está descartando informações relevantes, reforçando a robustez da abordagem automatizada.

- O ajuste leve de hiperparâmetros apresentou ganhos consistentes de performance, elevando o ROC AUC para aproximadamente 0.70. Esse resultado demonstra que, mesmo em um dataset de grande escala, **tuning controlado e bem delimitado** pode trazer benefícios reais, sem incorrer em custos computacionais excessivos ou risco elevado de overfitting.

- Por fim, o desempenho do modelo Naive Bayes, semelhante ao da Regressão Logística e significativamente inferior ao LightGBM, confirma que o problema não é trivial e não pode ser explicado por relações estatísticas simples entre as variáveis. A capacidade de modelar interações complexas é fundamental para alcançar bons resultados.

- De forma geral, os experimentos indicam que **modelos baseados em árvores, com tuning leve e sem reamostragem**, oferecem o melhor equilíbrio entre performance, robustez e custo computacional para este problema.


#### Pipeline

In [ ]:
# est1 = automlx.Pipeline(task='classification', score_metric='recall_macro') # type: ignore
# est1.fit(X_train, y_train)

### Avaliação do Modelo

In [ ]:
# evaluate_model(est1, X_test, y_test)

### Análise do Processo de otimização do AutoMLx

Nesta seção analisamos o processo de otimização executado pelo AutoMLx de forma a compreender melhor o processo de modelagem e garantir a qualidade do resultado obtido.  

Para mais informações acesse:
- [Documentação do AutoMLx](https://docs.oracle.com/en-us/iaas/tools/automlx/latest/latest/index.html)  
- [Demo: AutoMLx Classification](https://github.com/oracle-samples/automlx/blob/main/demos/OracleAutoMLx_Classification.ipynb)

#### Sumário

Sumário dos experimentos executados pelo AutoMLx.

In [ ]:
# est1.print_summary()

#### Visualizações

Cada experimento (`trial`) nos gráficos abaixo é uma linha em um dataframe que contém:  
Algoritmo, Número de Amostras, Número de Features, Hiperparâmetros, Score, Tempo de Execução, Uso de Memória e Etapa.  

Para saber mais:
- [Acesse as funções](./utils/plot_automl_results.py)  

- Utilize o método `help()` para cada função.  
    <small>Exemplo: "help(plot_algorithm_selection)"</small>

##### Seleção de Algoritmo

Processo de seleção de algoritmo do AutoMLx.
- `Y`: Métrica
- `X`: Modelo

In [ ]:
# plot_algorithm_selection(est1)

##### Sampling Adaptativo

Processo de seleção de amostragem do AutoMLx.
- `Y`: Métrica
- `X`: Tamanho da Amostra

In [ ]:
# plot_adaptive_sampling(est1)

##### Seleção de Features

Processo de seleção de features do AutoMLx.
- `Y`: Métrica
- `X`: Número de features

In [ ]:
# plot_feature_selection(est1, df.drop(columns=['Atrasado']))

##### Ajuste de Hiperparâmetros

Processo de seleção de hiperparâmetros do AutoMLx.
- `Y`: Métrica
- `X`: Iteração *n*

In [ ]:
# plot_model_tuning(est1)

### Exportação do Modelo

Aqui exportamos o modelo em formato `.pkl` para uso em inferência.

In [ ]:
# best_estimator = est1
# model_metadata = {
#     'model': best_estimator.selected_model_,
#     'specifications': best_estimator.selected_model_params_,
#     'features': best_estimator.selected_features_names_,
#     'features_raw': list(set(best_estimator.selected_features_names_raw_)),
# }
# print(model_metadata)

In [ ]:
# export_model(best_estimator, filename=f'flight_delay_{model_metadata["model"]}', timestamp=True)

In [ ]:
# document_model(best_estimator, model_name=model_metadata['model'], model_metadata=model_metadata, model_features=model_metadata['features_raw'], x=X_test, y=y_test, output_format='.md', timestamp=True)

#### Testando o modelo exportado

In [ ]:
model_filename = 'flight_delay_XGBClassifier_20260108_173529.pkl'
# test_model(model_filename, X_test)